In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 📁 저장할 폴더 지정
download_dir = os.path.abspath("hyundia_images")
os.makedirs(download_dir, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,           # 다운로드 경로
    "download.prompt_for_download": False,                # 다운로드 팝업 비활성화
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.hyundainews.com/en-us/gallery/images?model_id=&category_ids=328%2C328&event_ids=&image-categories=&models=&year=&advanced_search=&image-tag=Front+34&sort_by=published-desc"
driver.get(url)
time.sleep(2)

In [ ]:
page = 1
while True:
    print(f"\n=== 페이지 {page} 처리 중 ===")
    download_buttons = driver.find_elements(By.CSS_SELECTOR, "a:nth-child(3).js-basket-action--direct")
	# print(f"🔍 다운로드 버튼 {len(download_buttons)}개 발견")
    for i, download_button in enumerate(download_buttons[:2]):  # 처음 5개만 테스트
        try:
            driver.execute_script("arguments[0].click();", download_button)
            print(f"[{i+1}] 다운로드 버튼 클릭")
        except Exception as e:
            print(f"❌ [{i}] 클릭 실패: {e}")
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "body > div.content.js-content > div:nth-child(4) > section > div.pagination > nav.pagination__nav > ul > li:nth-child(7) > a")
        next_href = next_button.get_attribute("href")
        if not next_href:
            print("⛔ 다음 페이지 없음")
            break
        print("➡️ 다음 페이지로 이동:", next_href)
        driver.get(next_href)
        time.sleep(2)
        page += 1
    except Exception as e:
        print("❌ 다음 페이지 버튼 탐색 실패:", e)
        break

In [ ]:
# 누끼
import os
from rembg import remove
from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # <- 이 줄을 반드시 추가!

input_dir = "hyundai_images"
output_dir = "hyundai_images_nobg"
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if not filename.lower().endswith('.jpg'):
        continue

    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)

    with open(input_path, 'rb') as f:
        input_image = f.read()

    output_image = remove(input_image)  # 배경 제거
    with open(output_path, 'wb') as f:
        f.write(output_image)

    print(f"✅ 누끼 완료: {filename}")

In [ ]:
# 누끼
import os
from rembg import remove
from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # <- 이 줄을 반드시 추가!

input_dir = "hyundai_concept_car"
output_dir = "hyundai_concept_car_nobg"
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if not filename.lower().endswith('.jpg'):
        continue

    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)

    with open(input_path, 'rb') as f:
        input_image = f.read()

    output_image = remove(input_image)  # 배경 제거
    with open(output_path, 'wb') as f:
        f.write(output_image)

    print(f"✅ 누끼 완료: {filename}")

In [ ]:
# Focus crop
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

def get_foreground_bbox(img, bg_color=(0,0,0), tolerance=10):
    """검정색 배경에서 객체의 bounding box 좌표 추출 (tolerance로 어두운 부분 허용)"""
    if img.mode != 'RGB':
        img = img.convert('RGB')
    np_img = np.array(img)
    # mask: 배경이 아닌 영역 True
    mask = np.any(np.abs(np_img - bg_color) > tolerance, axis=2)
    coords = np.argwhere(mask)
    if coords.shape[0] == 0:
        # 객체가 아예 없으면 전체
        return (0, 0, img.width, img.height)
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0)
    return (x0, y0, x1+1, y1+1)  # (left, top, right, bottom)

def focus_crop_and_pad(img, target_size=1024, bg_color=(255,255,255), tolerance=10):
    """객체 감지 후 crop, 중앙 정렬 pad, 1024x1024 resize"""
    bbox = get_foreground_bbox(img, bg_color, tolerance)
    cropped = img.crop(bbox)
    # 정사각형으로 중앙 패딩
    max_side = max(cropped.width, cropped.height)
    square_img = Image.new('RGB', (max_side, max_side), bg_color)
    left = (max_side - cropped.width) // 2
    top = (max_side - cropped.height) // 2
    square_img.paste(cropped, (left, top))
    # 1024로 resize
    out_img = square_img.resize((target_size, target_size), Image.LANCZOS)
    return out_img

# 경로 설정
input_dir = './images/hyundai_concept_car_nogb'   # 원본 이미지 폴더
output_dir = './images/hyundai_concept_car_nogb_cropped_1024'  # 저장할 폴더
os.makedirs(output_dir, exist_ok=True)

img_exts = ['.jpg', '.jpeg', '.png', '.bmp']

for fname in tqdm(os.listdir(input_dir)):
    ext = os.path.splitext(fname)[-1].lower()
    if ext not in img_exts:
        continue
    img_path = os.path.join(input_dir, fname)
    out_path = os.path.join(output_dir, os.path.splitext(fname)[0] + '.jpg')
    try:
        img = Image.open(img_path)
        cropped_img = focus_crop_and_pad(img, target_size=1024, bg_color=(255,255,255), tolerance=10)
        cropped_img.save(out_path)
    except Exception as e:
        print(f'Error processing {fname}: {e}')


100%|██████████| 56/56 [00:02<00:00, 27.61it/s]


In [18]:
# focus crop + resize

import os
from PIL import Image
import numpy as np
from tqdm import tqdm

def get_alpha_bbox(img, alpha_threshold=10):
    """알파 채널로 비투명 객체 bbox 추출"""
    if img.mode != 'RGBA':
        img = img.convert('RGBA')
    alpha = np.array(img.split()[-1])
    mask = alpha > alpha_threshold
    coords = np.argwhere(mask)
    if coords.shape[0] == 0:
        return (0, 0, img.width, img.height)
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0)
    return (x0, y0, x1+1, y1+1)  # left, top, right, bottom

def focus_crop_pad_and_merge(img, target_size=1024, bg_color=(255,255,255)):
    """알파 영역 감지 → 중심정렬 패딩 → RGB 배경 합성 → 1024x1024 리사이즈"""
    bbox = get_alpha_bbox(img)
    cropped = img.crop(bbox)
    max_side = max(cropped.width, cropped.height)
    # 1. 배경색 RGB 이미지 만들기
    bg_rgb = Image.new('RGB', (max_side, max_side), bg_color)
    # 2. 중심에 객체 붙여넣기 (알파 마스킹)
    left = (max_side - cropped.width) // 2
    top = (max_side - cropped.height) // 2
    bg_rgb.paste(cropped, (left, top), mask=cropped.split()[-1])
    # 3. 최종 1024x1024 리사이즈
    out_img = bg_rgb.resize((target_size, target_size), Image.LANCZOS)
    return out_img

# 경로 설정
input_dir = './images/hyundai_images_nobg'        # 누끼 PNG 이미지 폴더
output_dir = './images/hyundai_images_nobg_cropped_1024'             # 변환된 결과 저장 폴더
os.makedirs(output_dir, exist_ok=True)

for fname in tqdm(os.listdir(input_dir)):
    if not fname.lower().endswith('.jpg'):
        continue
    path = os.path.join(input_dir, fname)
    out_path = os.path.join(output_dir, os.path.splitext(fname)[0] + '.jpg')  # RGB로 저장(JPG)
    try:
        img = Image.open(path)
        result = focus_crop_pad_and_merge(img, target_size=1024, bg_color=(255,255,255))  # 밝은 회색
        result.save(out_path, quality=100)
    except Exception as e:
        print(f"❌ {fname} 에러: {e}")


 39%|███▊      | 669/1728 [11:26<13:49,  1.28it/s]  c:\Users\Playdata\miniconda3\envs\babsim\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (101082464 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 56%|█████▌    | 969/1728 [16:36<20:57,  1.66s/it]c:\Users\Playdata\miniconda3\envs\babsim\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (115642240 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 68%|██████▊   | 1167/1728 [20:58<15:30,  1.66s/it]c:\Users\Playdata\miniconda3\envs\babsim\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (101896752 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 87%|████████▋ | 1511/1728 [26:46<05:16,  1.46s/it]c:\Users\Playdata\miniconda3\envs\babsim\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (10377

❌ Large-66579-2026ElantraHybrid.jpg 에러: Image size (207783696 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.


100%|██████████| 1728/1728 [32:04<00:00,  1.11s/it]
